# Using Ultralytics YOLOv9, ByteTrack and Supervision for tracking and counting



    Detection with yolov9
    
    Tracking with BoT-SORT
    
    Counting using supervision
    

In [2]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%cd /content/drive/MyDrive
!mkdir Yolov8_Vehical_Counting
%cd /content/drive/MyDrive/Yolov8_Vehical_Counting

/content/drive/MyDrive
/content/drive/MyDrive/Yolov8_Vehical_Counting


In [13]:
!pip install ultralytics

import ultralytics
ultralytics.__version__

'8.3.146'

In [15]:
!pip install supervision
import supervision
print("supervision.__version__:", supervision.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 3.2 MB/s eta 0:00:00
supervision.__version__: 0.25.1


In [16]:
import torch
torch.__version__

'2.6.0+cu124'

In [24]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Detection

In [25]:
!mkdir inference

In [26]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Run inference on 'bus.jpg' with arguments
model.predict(source="inference/img.png", save=True, imgsz=320, conf=0.5)

100%|██████████| 6.25M/6.25M [00:00<00:00, 51.5MB/s]



image 1/1 /content/drive/MyDrive/Yolov8_Vehical_Counting/inference/img.png: 192x320 3 cars, 208.8ms
Speed: 9.2ms preprocess, 208.8ms inference, 24.9ms postprocess per image at shape (1, 3, 192, 320)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

# Tracking

In [27]:
from ultralytics import YOLO

# Configure the tracking parameters and run the tracker
model = YOLO('yolov8n.pt')

results = model.track(source="inference/d.mp4",conf=0.3, iou=0.5, save=True, tracker="bytetrack.yaml")

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/687) /content/drive/MyDrive/Yolov8_Vehical_Counting/inference/d.mp4: 384x640 9 cars, 1 stop sign, 163.9ms
video 1/1 (frame 2/687) /content/drive/MyDrive/Yolov8_Vehical_Counting/inference/d.mp4: 384x640 9 cars, 1 stop sig

KeyboardInterrupt: 

In [29]:
!pip install --upgrade supervision

# Tracking and counting  - 1 line

In [31]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Set up video capture
cap = cv2.VideoCapture("inference/d.mp4")

# Define the line coordinates
START = sv.Point(182, 254)
END = sv.Point(462, 254)


# Store the track history
track_history = defaultdict(lambda: [])

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects = {}

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path("inference/d.mp4")
with sv.VideoSink("output_single_line.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, classes=[2, 3, 5, 7], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            #detections = sv.Detections.from_yolov8(results[0]) #For this line code i replace below lines
            boxes = results[0].boxes.xyxy.cpu().numpy()      # bounding boxes in (x1, y1, x2, y2)
            scores = results[0].boxes.conf.cpu().numpy()     # confidence scores
            classes = results[0].boxes.cls.cpu().numpy().astype(int)  # class IDs

            detections = sv.Detections(
                xyxy=boxes,
                confidence=scores,
                class_id=classes
                )


            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Check if the object crosses the line
                if START.x < x < END.x and abs(y - START.y) < 5:  # Assuming objects cross horizontally
                    if track_id not in crossed_objects:
                        crossed_objects[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)

            # Draw the line on the frame
            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 255, 0), 2)

            # Write the count of objects on each frame
            count_text = f"Objects crossed: {len(crossed_objects)}"
            cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()



0: 384x640 10 cars, 195.2ms
Speed: 2.8ms preprocess, 195.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 10 cars, 195.9ms
Speed: 2.1ms preprocess, 195.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 10 cars, 181.5ms
Speed: 2.1ms preprocess, 181.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 9 cars, 192.5ms
Speed: 2.1ms preprocess, 192.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 9 cars, 197.3ms
Speed: 2.4ms preprocess, 197.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 9 cars, 191.0ms
Speed: 2.1ms preprocess, 191.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 9 cars, 216.3

KeyboardInterrupt: 